### LLM을 활용한 Context 보완 및 예상 질문 생성 작업
#### data_preprocessing.ipynb를 통해 나온 final_documents.pkl 전처리

#### 1. 초기 설정 및 라이브러리 임포트

In [13]:
import pickle
import os
from tqdm import tqdm
from datetime import datetime
from typing import List, Dict
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

# 중간 저장을 위한 디렉토리 설정
CHECKPOINT_DIR = "./data/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

#### 2. 데이터 로드 및 초기 상태 확인

In [14]:
def load_and_verify_data(filepath: str):
    """데이터 로드 및 초기 상태 확인"""
    with open(filepath, 'rb') as f:
        documents = pickle.load(f)
    
    # 초기 상태 분석
    total_docs = len(documents)
    category_counts = {}
    for doc in documents:
        cat = doc.metadata.get('category', 'unknown')
        category_counts[cat] = category_counts.get(cat, 0) + 1
    
    print(f"총 문서 수: {total_docs}")
    print("\n카테고리별 문서 수:")
    for cat, count in category_counts.items():
        print(f"{cat}: {count}")
    
    return documents

# 실행
documents = load_and_verify_data("./data/final_documents.pkl")

총 문서 수: 3629

카테고리별 문서 수:
table: 1086
paragraph: 1214
footer: 306
chart: 598
list: 251
figure: 167
caption: 7


#### 3. Summary 생성 Agent 설정

In [22]:
def create_summary_agent():
    """Summary 생성을 위한 Agent 설정"""
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )
    
    summary_prompt = """
    다음 내용에 대한 요약을 생성해주세요. 
    요약은 RAG 시스템에서 효과적으로 검색될 수 있도록 작성되어야 합니다.
    
    규칙:
    1. 테이블, 차트, 이미지의 정보가 핵심적으로 드러나야 합니다.
    2. 핵심 키워드와 정확한 수치 포함(수치 주의! '십억원', '억원', '천원' 등)
    3. 객관적 사실 중심
    
    내용:
    {content}
    
    요약:
    """
    
    return llm, summary_prompt

#### 4. Summary 생성 및 추가 프로세스:

In [23]:
def process_summaries(documents: List[Document], batch_size: int = 5):
    """문서 요약 처리 및 메타데이터 추가"""
    llm, prompt = create_summary_agent()
    
    # 처리가 필요한 문서만 필터링
    target_categories = ['table', 'chart', 'figure']
    docs_to_process = [doc for doc in documents if doc.metadata.get('category') in target_categories]
    processed_count = len(docs_to_process)
    
    print(f"처리 대상 문서 수: {processed_count} (table: {sum(1 for doc in docs_to_process if doc.metadata.get('category')=='table')}, "
          f"chart: {sum(1 for doc in docs_to_process if doc.metadata.get('category')=='chart')}, "
          f"figure: {sum(1 for doc in docs_to_process if doc.metadata.get('category')=='figure')})")
    
    current_doc_idx = 0
    
    # 진행상황 표시를 위한 tqdm 설정
    for i in tqdm(range(0, len(docs_to_process), batch_size), desc="Summary 생성"):
        batch = docs_to_process[i:i+batch_size]
        
        for doc in batch:
            try:
                # Summary 생성
                response = llm.invoke(prompt.format(content=doc.page_content))
                summary = response.content
                
                # 메타데이터에 summary 추가
                new_metadata = doc.metadata.copy()
                new_metadata['summary'] = summary
                
                # 원본 documents 리스트에서 해당 문서 찾아 업데이트
                while current_doc_idx < len(documents):
                    if documents[current_doc_idx].metadata.get('uuid') == doc.metadata.get('uuid'):
                        documents[current_doc_idx] = Document(
                            page_content=doc.page_content,
                            metadata=new_metadata
                        )
                        current_doc_idx += 1
                        break
                    current_doc_idx += 1
                    
            except Exception as e:
                print(f"\n에러 발생: {e}")
        
        # 배치 처리 후 중간 저장
        if (i + batch_size) % 50 == 0:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            checkpoint_path = os.path.join(CHECKPOINT_DIR, f"summary_checkpoint_{timestamp}.pkl")
            with open(checkpoint_path, 'wb') as f:
                pickle.dump(documents, f)
            print(f"\n중간 저장 완료: {checkpoint_path}")
    
    return documents

#### 5. Question Generation Agent 설정 및 실행

In [24]:
def create_question_agent():
    """질문 생성을 위한 Agent 설정"""
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7
    )
    
    question_prompt = """
    주어진 내용을 바탕으로 4개의 질문을 생성해주세요.
    
    난이도 분포:
    - 쉬움 (1개): 직접적인 정보 추출
    - 중간 (2개): 정보 조합 필요
    - 어려움 (1개): 분석/추론 필요
    
    답변은 반드시 주어진 내용에서 도출 가능해야 합니다.
    
    내용:
    {content}
    
    질문 목록:
    """
    
    return llm, question_prompt

def process_questions(documents: List[Document], batch_size: int = 5):
    """질문 생성 및 메타데이터 추가"""
    llm, prompt = create_question_agent()
    processed_docs = []
    
    for i in tqdm(range(0, len(documents), batch_size), desc="질문 생성"):
        batch = documents[i:i+batch_size]
        
        for doc in batch:
            try:
                # 질문 생성
                response = llm.invoke(prompt.format(content=doc.page_content))
                questions = response.content.split('\n')
                
                # 메타데이터에 질문 추가
                new_metadata = doc.metadata.copy()
                new_metadata['expected_questions'] = questions
                
                processed_docs.append(Document(
                    page_content=doc.page_content,
                    metadata=new_metadata
                ))
            except Exception as e:
                print(f"에러 발생: {e}")
                processed_docs.append(doc)
        
        # 중간 저장
        if (i + batch_size) % 50 == 0:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            checkpoint_path = os.path.join(CHECKPOINT_DIR, f"questions_checkpoint_{timestamp}.pkl")
            with open(checkpoint_path, 'wb') as f:
                pickle.dump(processed_docs, f)
            print(f"\n중간 저장 완료: {checkpoint_path}")
    
    return processed_docs

#### 6. 최종 저장 및 검증

In [ ]:
def save_and_verify_final_documents(documents: List[Document]):
    """최종 문서 저장 및 검증"""
    # 최종 저장
    final_path = "./data/final_documents_ver2.pkl"
    with open(final_path, 'wb') as f:
        pickle.dump(documents, f)
    
    # 검증
    print("\n=== 최종 검증 결과 ===")
    
    # 문서 수 확인
    total_docs = len(documents)
    print(f"총 문서 수: {total_docs}")
    
    # 메타데이터 필드 확인
    summary_count = sum(1 for doc in documents if 'summary' in doc.metadata)
    questions_count = sum(1 for doc in documents if 'expected_questions' in doc.metadata)
    
    print(f"Summary 포함 문서 수: {summary_count}")
    print(f"Questions 포함 문서 수: {questions_count}")
    
    # 샘플 출력
    if documents:
        print("\n=== 샘플 문서 확인 ===")
        sample_doc = documents[0]
        print(f"카테고리: {sample_doc.metadata.get('category')}")
        print(f"Summary: {sample_doc.metadata.get('summary', 'None')[:200]}...")
        print("\nExpected Questions:")
        for q in sample_doc.metadata.get('expected_questions', [])[:2]:
            print(f"- {q}")
    
    return final_path

# 전체 프로세스 실행
documents_with_summary = process_summaries(documents)
documents_with_questions = process_questions(documents_with_summary)
final_path = save_and_verify_final_documents(documents_with_questions)

### 위의 작업이 끝나면 끝나면 아래와 같은 것을 실험해볼 수 있습니다.

- (./data/final_documents_ver2.pkl)데이터 로드 한 후

1. 메타데이터에 있는 summary를 page_content 뒷단에 추가 
2. 메타데이터에 있는 summary를 page_content 뒷단에 추가 + 예상질문도 추가

#### 1번 실험을 위한 pkl 파일 생성 코드

In [ ]:
filepath = "./data/final_documents_ver2.pkl"

with open(filepath, 'rb') as f:
    documents_ver2 = pickle.load(f)

In [ ]:
def enhance_content_with_summary(documents: List[Document]) -> List[Document]:
   """
   table, chart, figure 문서의 page_content에 summary 정보만 추가
   """
   enhanced_documents = []
   
   for doc in documents:
       # 원본 content 저장
       original_content = doc.page_content
       new_content_parts = []
       
       # Summary 추가 (table, chart, figure인 경우만)
       if doc.metadata.get('category') in ['table', 'chart', 'figure'] and 'summary' in doc.metadata:
           new_content_parts.append(f"Summary:\n{doc.metadata['summary']}\n")
       
       # 원본 content 추가
       new_content_parts.append(original_content)
       
       # 새로운 Document 생성
       enhanced_doc = Document(
           page_content="\n".join(new_content_parts),
           metadata=doc.metadata
       )
       enhanced_documents.append(enhanced_doc)
   
   # 처리 결과 출력
   total_docs = len(enhanced_documents)
   docs_with_summary = sum(1 for doc in enhanced_documents if doc.metadata.get('category') in ['table', 'chart', 'figure'] and 'summary' in doc.metadata)
   
   print(f"총 처리된 문서 수: {total_docs}")
   print(f"Summary가 추가된 문서 수: {docs_with_summary}")
   
   # 샘플 출력 (summary가 있는 문서 중에서)
   if docs_with_summary > 0:
       print("\n=== Summary가 추가된 문서 샘플 ===")
       sample_doc = next(doc for doc in enhanced_documents 
                       if doc.metadata.get('category') in ['table', 'chart', 'figure'] 
                       and 'summary' in doc.metadata)
       print(f"\n카테고리: {sample_doc.metadata.get('category')}")
       print(f"Content 시작 부분:\n{sample_doc.page_content[:500]}...")
   
   return enhanced_documents

# 실행
enhanced_docs = enhance_content_with_summary(documents_ver2)

# 새로운 pickle 파일로 저장
output_path = "./data/final_documents_ver2_with_summary.pkl"
with open(output_path, 'wb') as f:
   pickle.dump(enhanced_docs, f)

print(f"\n결과 저장 완료: {output_path}")

#### 2번 실험을 위한 pkl 파일 생성 코드

In [ ]:
filepath = "./data/final_documents_ver2.pkl"

with open(filepath, 'rb') as f:
    documents_ver2 = pickle.load(f)

In [ ]:
def enhance_page_content(documents: List[Document]) -> List[Document]:
    """
    모든 문서의 page_content에 summary와 expected_questions 정보 추가
    """
    enhanced_documents = []
    
    for doc in documents:
        # 원본 content 저장
        original_content = doc.page_content
        new_content_parts = []
        
        # Summary 추가 (table, chart, figure인 경우)
        if doc.metadata.get('category') in ['table', 'chart', 'figure'] and 'summary' in doc.metadata:
            new_content_parts.append(f"Summary:\n{doc.metadata['summary']}\n")
        
        # Expected Questions 추가
        if 'expected_questions' in doc.metadata:
            new_content_parts.append("Expected Questions:")
            for question in doc.metadata['expected_questions']:
                new_content_parts.append(f"- {question}")
            new_content_parts.append("")  # 빈 줄 추가
        
        # 원본 content 추가
        new_content_parts.append(f"Original Content:\n{original_content}")
        
        # 새로운 Document 생성
        enhanced_doc = Document(
            page_content="\n".join(new_content_parts),
            metadata=doc.metadata
        )
        enhanced_documents.append(enhanced_doc)
    
    # 처리 결과 출력
    total_docs = len(enhanced_documents)
    docs_with_summary = sum(1 for doc in enhanced_documents if doc.metadata.get('category') in ['table', 'chart', 'figure'] and 'summary' in doc.metadata)
    docs_with_questions = sum(1 for doc in enhanced_documents if 'expected_questions' in doc.metadata)
    
    print(f"총 처리된 문서 수: {total_docs}")
    print(f"Summary가 추가된 문서 수: {docs_with_summary}")
    print(f"Expected Questions가 추가된 문서 수: {docs_with_questions}")
    
    # 샘플 출력
    if enhanced_documents:
        print("\n=== 처리된 문서 샘플 ===")
        sample_doc = next(doc for doc in enhanced_documents if 'expected_questions' in doc.metadata)
        print(f"\n카테고리: {sample_doc.metadata.get('category')}")
        print(f"Content 시작 부분:\n{sample_doc.page_content[:500]}...")
    
    return enhanced_documents

# 실행
enhanced_docs = enhance_page_content(documents_ver2)

# 새로운 pickle 파일로 저장
output_path = "./data/final_documents_ver2_with_question_summary.pkl"
with open(output_path, 'wb') as f:
    pickle.dump(enhanced_docs, f)

print(f"\n결과 저장 완료: {output_path}")